#### Note: 
    Here main idea is to find class of an object based on encoded HTML features.

# Import libs

In [1]:
import torch


import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

# Import data

In [2]:
df = pd.read_parquet("../data/experiment_eda.parquet")

In [3]:
df.head()

,CLIENT_IP,EVENT_ID,HEADER_pattern,BOT_BOOL,CURL_HEAD_BOOL,WGET_HEAD_BOOL,SYMBOL_@,LENGTH_OF_USER_AGENT_HEAD_likely,windows_bool,linux_bool,...,REQUEST_URI\';\'REQUEST_ARGS,REQUEST_XML,RESPONSE_BODY,RESPONSE_HEADERS,2,3,4,5,Anomaly TOP,user_expected
0,93.158.215.131,AVdcJmIIq1Ppo9zF2YIp,True,False,False,False,False,True,True,False,...,False,False,False,False,False,True,False,False,False,True
1,176.123.240.64,iz7SN2YBrgKk_RFNZW_U,True,False,False,False,False,True,True,False,...,False,False,False,False,False,False,True,False,False,True
2,73.78.81.63,AVdjekw4q1Ppo9zF6QT2,True,False,False,False,False,True,True,False,...,False,False,False,False,False,True,False,False,False,True
3,66.102.9.44,SqQGI2QB5cBXmMW1CDbp,True,False,False,False,False,True,False,True,...,False,False,False,False,False,False,True,False,False,True
4,79.183.87.247,nFzwHGQB5cBXmMW1y_TD,True,False,False,False,False,True,False,True,...,False,False,False,False,False,False,True,False,False,True


# Import MLPAutoEncdoer

In [4]:
model = torch.jit.load('../models/baseline_mlp_autoencdoer.pt')

## Define Encoder function

In [5]:
@torch.inference_mode()
def encode_data(model, data) -> np.array:
    model.eval()
    return model.encoder(torch.Tensor(data[2:].astype(float)).unsqueeze(0).view(1,1,42)).view(10).numpy()

# Pipline to get encode list of features

In [7]:
list_of_features = []
for row in df.values:
    list_of_features.append(encode_data(model, np.array(row)))
list_of_features = np.array(list_of_features)

In [8]:
df_zip = pd.DataFrame(data=list_of_features)

# Load OPTICS auto_classification

In [11]:
from sklearn.cluster import OPTICS

In [18]:
clustering = OPTICS(min_samples=500).fit(df_zip)

/home/gishb/.local/lib/python3.10/site-packages/sklearn/cluster/_optics.py:995: RuntimeWarning: divide by zero encountered in divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]


In [20]:
unique_classes = np.unique(clustering.labels_)

In [22]:
df['class'] = clustering.labels_

# Save OPTICS classification

In [30]:
df[["EVENT_ID","class"]].to_parquet("../data/event_classes_by_optics_50")

In [31]:
df[["EVENT_ID","class"]].head()

,EVENT_ID,class
0,AVdcJmIIq1Ppo9zF2YIp,0
1,iz7SN2YBrgKk_RFNZW_U,-1
2,AVdjekw4q1Ppo9zF6QT2,0
3,SqQGI2QB5cBXmMW1CDbp,-1
4,nFzwHGQB5cBXmMW1y_TD,39
